In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# sqrtm

> Methods for computing sqrt of a matrix

In [ ]:
#| default_exp sqrtm

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch
from torch.autograd import Function, Variable

## Steve Li's method

In [ ]:
#| export

use_li = False  # come back and turn this on if you want to see/use the full code

if use_li:      # lighten the load of imports since we won't use li's in production
    import numpy as np
    import scipy.linalg

In [ ]:
#| export
class MatrixSquareRoot_li(Function):
    """
    From https://github.com/steveli/pytorch-sqrtm/blob/master/sqrtm.py, which sadly does not install as a package.  LICENSE included below
    Square root of a positive definite matrix.

    NOTE: matrix square root is not differentiable for matrices with
          zero eigenvalues.
    """
    @staticmethod
    def forward(ctx, input):
        m = input.detach().cpu().numpy().astype(np.float_)  # SHH: note how this immediately switches to CPU & numpy :-( 
        sqrtm = torch.from_numpy(scipy.linalg.sqrtm(m).real).to(input)
        ctx.save_for_backward(sqrtm)
        return sqrtm

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = None
        if ctx.needs_input_grad[0]:
            sqrtm, = ctx.saved_tensors
            sqrtm = sqrtm.data.cpu().numpy().astype(np.float_)
            gm = grad_output.data.cpu().numpy().astype(np.float_)

            # Given a positive semi-definite matrix X,
            # since X = X^{1/2}X^{1/2}, we can compute the gradient of the
            # matrix square root dX^{1/2} by solving the Sylvester equation:
            # dX = (d(X^{1/2})X^{1/2} + X^{1/2}(dX^{1/2}).
            grad_sqrtm = scipy.linalg.solve_sylvester(sqrtm, sqrtm, gm)

            grad_input = torch.from_numpy(grad_sqrtm).to(grad_output)
        return grad_input


sqrtm_li = MatrixSquareRoot_li.apply


LICENCE_LI = """
MIT License

Copyright (c) 2022 Steven Cheng-Xian Li

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

Steve Li's test code for the above:


In [ ]:
#| eval: false 
from torch.autograd import gradcheck

In [ ]:
#| eval: false 
if use_li:
    k = torch.randn(20, 10).double()
    # Create a positive definite matrix
    pd_mat = (k.t().matmul(k)).requires_grad_()
    sq = sqrtm_li(pd_mat)
    print(sq)
    test = gradcheck(sqrtm_li, (pd_mat,))
    print(test)

tensor([[ 4.3880e+00,  3.0241e-01, -1.3668e-01,  5.1494e-01, -5.3276e-01,
          2.7611e-01, -8.6985e-01, -5.1909e-01,  7.9893e-01,  4.4863e-01],
        [ 3.0241e-01,  4.8187e+00,  3.3622e-01,  4.6490e-01,  5.7077e-01,
         -4.6965e-01, -1.3664e-01,  3.3543e-01, -8.7196e-01,  5.2538e-02],
        [-1.3668e-01,  3.3622e-01,  4.2170e+00, -1.3224e-01, -6.8163e-01,
          3.2648e-01,  6.0498e-01,  9.8877e-01, -3.9825e-01, -5.8457e-02],
        [ 5.1494e-01,  4.6490e-01, -1.3224e-01,  3.2379e+00,  6.9414e-01,
         -4.4017e-01, -1.9384e-01,  3.7950e-01, -2.5017e-03, -3.5434e-01],
        [-5.3276e-01,  5.7077e-01, -6.8163e-01,  6.9414e-01,  4.7020e+00,
          1.7619e-01, -9.9632e-02,  2.9081e-02, -2.6256e-01, -3.7996e-01],
        [ 2.7611e-01, -4.6965e-01,  3.2648e-01, -4.4017e-01,  1.7619e-01,
          4.5748e+00, -7.2716e-01,  2.6542e-02, -4.6386e-01,  6.8533e-02],
        [-8.6985e-01, -1.3664e-01,  6.0498e-01, -1.9384e-01, -9.9632e-02,
         -7.2716e-01,  3.9740e+0

## Subhransu Maji's method(s)

From https://github.com/msubhransu/matrix-sqrt

In [ ]:
#| export

# Compute error
def compute_error(A, sA):
    normA = torch.sqrt(torch.sum(torch.sum(A * A, dim=1),dim=1))
    error = A - torch.bmm(sA, sA)
    error = torch.sqrt((error * error).sum(dim=1).sum(dim=1)) / normA
    return torch.mean(error)


def sqrt_newton_schulz_autograd(A, 
                                numIters=7, # 7 found experimentally by SHH, comparing w/ Li's method
                               ):
    """Modified from from https://people.cs.umass.edu/~smaji/projects/matrix-sqrt/
    "The drawback of the autograd approach [i.e., this approach] is that a naive implementation stores all the intermediate results. 
    Thus the memory overhead scales linearly with the number of iterations which is problematic for large matrices."
    """
    if len(A.data.shape) < 3: A = A.unsqueeze(0)
    batchSize, dim1, dim2 = A.data.shape
    assert dim1==dim2
    dim, dtype, device = dim1, A.dtype, A.device
    normA = A.mul(A).sum(dim=1).sum(dim=1).sqrt()
    Y = A.div(normA.view(batchSize, 1, 1).expand_as(A));
    I = Variable(torch.eye(dim,dim, device=device).view(1, dim, dim).
                 repeat(batchSize,1,1).type(dtype),requires_grad=False)
    Z = Variable(torch.eye(dim,dim, device=device).view(1, dim, dim).
                 repeat(batchSize,1,1).type(dtype),requires_grad=False)

    for i in range(numIters):
        T = 0.5*(3.0*I - Z.bmm(Y))
        Y = Y.bmm(T)
        Z = T.bmm(Z)
    
    sA = Y*torch.sqrt(normA).view(batchSize, 1, 1).expand_as(A)
    error = compute_error(A, sA)
    return sA, error


def sqrt_newton_schulz(A,               # matrix to be sqrt-ified
                       numIters=7,      # numIters=7 found via experimentation
                       calc_error=True, # setting False disables Maji's error reporting
                      ):
    """
    Sqrt of matrix via Newton-Schulz algorithm
    Modified from https://github.com/msubhransu/matrix-sqrt/blob/cc2289a3ed7042b8dbacd53ce8a34da1f814ed2f/matrix_sqrt.py#LL72C1-L87C19
    # Forward via Newton-Schulz iterations (non autograd version)
    # Seems to be slighlty faster and has much lower memory overhead
    
    ... Original code didn't preserve device, had no batch dim checking -SHH
    """
    while len(A.data.shape) < 3:  # needs a batch dimension
        A = A.unsqueeze(0)
    batchSize, dim1, dim2 = A.data.shape
    assert dim1==dim2
    dim, dtype, device = dim1, A.dtype, A.device
    normA = A.mul(A).sum(dim=1).sum(dim=1).sqrt()
    Y = A.div(normA.view(batchSize, 1, 1).expand_as(A));
    I = torch.eye(dim,dim, device=device, dtype=dtype).view(1, dim, dim).repeat(batchSize,1,1)
    Z = torch.eye(dim,dim, device=device, dtype=dtype).view(1, dim, dim).repeat(batchSize,1,1)
    for i in range(numIters):
        T = 0.5*(3.0*I - Z.bmm(Y))
        Y = Y.bmm(T)
        Z = T.bmm(Z)
        
    sA = Y*torch.sqrt(normA).view(batchSize, 1, 1).expand_as(A)
    if calc_error: 
        error = compute_error(A, sA)
        return sA, error
    else:
        return sA

    
""" 
# Only used if backprop needed, which it isn't for FAD.  Leaving it here anyway. -SHH
def lyap_newton_schulz(z, dldz, numIters, dtype):
    # Backward via iterative Lyapunov solver.
    batchSize = z.shape[0]
    dim = z.shape[1]
    normz = z.mul(z).sum(dim=1).sum(dim=1).sqrt()
    a = z.div(normz.view(batchSize, 1, 1).expand_as(z))
    I = torch.eye(dim,dim).view(1, dim, dim).repeat(batchSize,1,1).type(dtype)
    q = dldz.div(normz.view(batchSize, 1, 1).expand_as(z))
    for i in range(numIters):
        q = 0.5*(q.bmm(3.0*I - a.bmm(a)) - a.transpose(1, 2).bmm(a.transpose(1,2).bmm(q) - q.bmm(a)) )
        a = 0.5*a.bmm(3.0*I - a.bmm(a))
    dlda = 0.5*q
    return dlda
"""


LICENCE_SM = """
MIT License

Copyright (c) 2017 Subhransu Maji

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

## Error tests

In [ ]:
#| eval: false 
sa1, error = sqrt_newton_schulz_autograd( pd_mat.unsqueeze(0), numIters=7 ) 
sa1, error

(tensor([[[ 4.3880e+00,  3.0243e-01, -1.3667e-01,  5.1496e-01, -5.3276e-01,
            2.7612e-01, -8.6987e-01, -5.1911e-01,  7.9897e-01,  4.4864e-01],
          [ 3.0243e-01,  4.8187e+00,  3.3622e-01,  4.6489e-01,  5.7077e-01,
           -4.6965e-01, -1.3664e-01,  3.3544e-01, -8.7198e-01,  5.2533e-02],
          [-1.3667e-01,  3.3622e-01,  4.2170e+00, -1.3225e-01, -6.8163e-01,
            3.2647e-01,  6.0499e-01,  9.8879e-01, -3.9827e-01, -5.8464e-02],
          [ 5.1496e-01,  4.6489e-01, -1.3225e-01,  3.2379e+00,  6.9414e-01,
           -4.4017e-01, -1.9383e-01,  3.7951e-01, -2.5238e-03, -3.5434e-01],
          [-5.3276e-01,  5.7077e-01, -6.8163e-01,  6.9414e-01,  4.7020e+00,
            1.7619e-01, -9.9634e-02,  2.9078e-02, -2.6255e-01, -3.7996e-01],
          [ 2.7612e-01, -4.6965e-01,  3.2647e-01, -4.4017e-01,  1.7619e-01,
            4.5748e+00, -7.2716e-01,  2.6549e-02, -4.6387e-01,  6.8529e-02],
          [-8.6987e-01, -1.3664e-01,  6.0499e-01, -1.9383e-01, -9.9634e-02,
      

In [ ]:
#| eval: false 
sa2, error = sqrt_newton_schulz( pd_mat.unsqueeze(0), numIters=7 ) 
sa2, error

(tensor([[[ 4.3880e+00,  3.0243e-01, -1.3667e-01,  5.1496e-01, -5.3276e-01,
            2.7612e-01, -8.6987e-01, -5.1911e-01,  7.9897e-01,  4.4864e-01],
          [ 3.0243e-01,  4.8187e+00,  3.3622e-01,  4.6489e-01,  5.7077e-01,
           -4.6965e-01, -1.3664e-01,  3.3544e-01, -8.7198e-01,  5.2533e-02],
          [-1.3667e-01,  3.3622e-01,  4.2170e+00, -1.3225e-01, -6.8163e-01,
            3.2647e-01,  6.0499e-01,  9.8879e-01, -3.9827e-01, -5.8464e-02],
          [ 5.1496e-01,  4.6489e-01, -1.3225e-01,  3.2379e+00,  6.9414e-01,
           -4.4017e-01, -1.9383e-01,  3.7951e-01, -2.5238e-03, -3.5434e-01],
          [-5.3276e-01,  5.7077e-01, -6.8163e-01,  6.9414e-01,  4.7020e+00,
            1.7619e-01, -9.9634e-02,  2.9078e-02, -2.6255e-01, -3.7996e-01],
          [ 2.7612e-01, -4.6965e-01,  3.2647e-01, -4.4017e-01,  1.7619e-01,
            4.5748e+00, -7.2716e-01,  2.6549e-02, -4.6387e-01,  6.8529e-02],
          [-8.6987e-01, -1.3664e-01,  6.0499e-01, -1.9383e-01, -9.9634e-02,
      

In [ ]:
#| eval: false 
if use_li:
    sa1 - sq

tensor([[[-2.6899e-05,  1.2056e-05,  1.4747e-05,  1.5928e-05, -3.1680e-06,
           7.2472e-06, -1.5690e-05, -2.3918e-05,  3.7398e-05,  1.2399e-05],
         [ 1.2056e-05, -5.4067e-06, -6.6034e-06, -7.1417e-06,  1.4228e-06,
          -3.2535e-06,  7.0253e-06,  1.0715e-05, -1.6765e-05, -5.5589e-06],
         [ 1.4747e-05, -6.6034e-06, -8.1006e-06, -8.7264e-06,  1.7300e-06,
          -3.9610e-06,  8.6202e-06,  1.3126e-05, -2.0497e-05, -6.7947e-06],
         [ 1.5928e-05, -7.1417e-06, -8.7264e-06, -9.4347e-06,  1.8788e-06,
          -4.2963e-06,  9.2842e-06,  1.4159e-05, -2.2148e-05, -7.3436e-06],
         [-3.1680e-06,  1.4228e-06,  1.7300e-06,  1.8788e-06, -3.7621e-07,
           8.5897e-07, -1.8400e-06, -2.8116e-06,  4.4074e-06,  1.4618e-06],
         [ 7.2472e-06, -3.2535e-06, -3.9610e-06, -4.2963e-06,  8.5897e-07,
          -1.9623e-06,  4.2133e-06,  6.4345e-06, -1.0081e-05, -3.3432e-06],
         [-1.5690e-05,  7.0253e-06,  8.6202e-06,  9.2842e-06, -1.8400e-06,
           4.2133e-

In [ ]:
#| eval: false 
if use_li:
    sa2 - sq

tensor([[[-2.6899e-05,  1.2056e-05,  1.4747e-05,  1.5928e-05, -3.1680e-06,
           7.2472e-06, -1.5690e-05, -2.3918e-05,  3.7398e-05,  1.2399e-05],
         [ 1.2056e-05, -5.4067e-06, -6.6034e-06, -7.1417e-06,  1.4228e-06,
          -3.2535e-06,  7.0253e-06,  1.0715e-05, -1.6765e-05, -5.5589e-06],
         [ 1.4747e-05, -6.6034e-06, -8.1006e-06, -8.7264e-06,  1.7300e-06,
          -3.9610e-06,  8.6202e-06,  1.3126e-05, -2.0497e-05, -6.7947e-06],
         [ 1.5928e-05, -7.1417e-06, -8.7264e-06, -9.4347e-06,  1.8788e-06,
          -4.2963e-06,  9.2842e-06,  1.4159e-05, -2.2148e-05, -7.3436e-06],
         [-3.1680e-06,  1.4228e-06,  1.7300e-06,  1.8788e-06, -3.7621e-07,
           8.5897e-07, -1.8400e-06, -2.8116e-06,  4.4074e-06,  1.4618e-06],
         [ 7.2472e-06, -3.2535e-06, -3.9610e-06, -4.2963e-06,  8.5897e-07,
          -1.9623e-06,  4.2133e-06,  6.4345e-06, -1.0081e-05, -3.3432e-06],
         [-1.5690e-05,  7.0253e-06,  8.6202e-06,  9.2842e-06, -1.8400e-06,
           4.2133e-

## Speed & device tests

In [ ]:
#| eval: false 
from aeiou.core import get_device

In [ ]:
#| eval: false 
device = get_device()
print('device = ',device)
n,m = 1000, 1000
with torch.no_grad(): 
    k = torch.randn(n, m, device=device)
    pd_mat2 = (k.t().matmul(k)) # Create a positive definite matrix, no grad

device =  cuda


In [ ]:
#| eval: false 
#%%timeit
if use_li:
    sq2 = sqrtm_li(pd_mat2)

Result of `%%timeit`:

`1.7 s ± 395 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)`

In [ ]:
#| eval: false 
#%%timeit
sq3 = sqrt_newton_schulz(pd_mat2.unsqueeze(0))

3.37 ms ± 8.44 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Result of `%%timeit`:

`13 ms ± 15.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)`

## Thus, we'll put a wrapper around our method of choice:

In [ ]:
#|export 
def sqrtm(A, numIters=7):
    "wrapper function for matrix sqrt algorithm of choice: sqrt_newton_schulz"
    return sqrt_newton_schulz(A, numIters=numIters, calc_error=False)

In [ ]:
#| eval: false  
sqrtm(pd_mat2) 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()